In [1]:
# Celda 1: Importaciones y Carga de Datos
import pandas as pd
import os

# Configurar pandas para una mejor visualización en el notebook
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', 150)
pd.set_option('display.max_rows', 100)

# --- 1. EJECUTAR EL SCRIPT DE ANÁLISIS PRINCIPAL ---
# Asegúrate de que la ruta al script sea la correcta desde la ubicación de tu notebook
script_path = '../scripts/ejecutar_analisis_clivajes_REU_06-08.py'

# Descomenta la siguiente línea para ejecutar el script desde el notebook si lo necesitas.
# Es mejor ejecutarlo desde la terminal si es un proceso largo.
# print("Ejecutando script de análisis... Esto puede tardar varios minutos.")
# %run {script_path}
# print("Script finalizado.")

# --- 2. CARGAR LOS RESULTADOS GENERADOS ---
results_dir = '../results'
try:
    df_terminos = pd.read_csv(os.path.join(results_dir, 'exp_analisis_terminos.csv'), sep=';')
    df_nodos = pd.read_csv(os.path.join(results_dir, 'exp_analisis_nodos.csv'), sep=';')
    df_metricas = pd.read_csv(os.path.join(results_dir, 'exp_metricas_polarizacion.csv'), sep=';')
    
    print("Archivos de resultados cargados exitosamente.")
    print(f"Términos encontrados: {len(df_terminos)} filas")
    print(f"Nodos centrales encontrados: {len(df_nodos)} filas")
    
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo de resultados. Asegúrate de haber ejecutado el script primero.")
    print(e)

Archivos de resultados cargados exitosamente.
Términos encontrados: 1304 filas
Nodos centrales encontrados: 75 filas


In [2]:
# Celda 2: Función para Mostrar Resultados de Forma Ordenada

def mostrar_analisis_completo(df_terminos, df_nodos, dataset, algoritmo, ngram_tipo):
    """
    Filtra y muestra una tabla comparativa con los top términos y nodos para una configuración específica.
    """
    
    print("="*80)
    print(f"Análisis para: DATASET={dataset} | ALGORITMO={algoritmo} | N-GRAMS={ngram_tipo}")
    print("="*80)
    
    # Filtrar datos para la configuración actual
    terminos_filtrados = df_terminos[
        (df_terminos['Dataset'] == dataset) & 
        (df_terminos['Algoritmo'] == algoritmo) & 
        (df_terminos['Tipo_Ngram'] == ngram_tipo)
    ]
    
    nodos_filtrados = df_nodos[
        (df_nodos['Dataset'] == dataset) & 
        (df_nodos['Algoritmo'] == algoritmo)
    ]
    
    if terminos_filtrados.empty or nodos_filtrados.empty:
        print("\nNo se encontraron datos para esta combinación.\n")
        return

    # Preparar DataFrames para la visualización
    # Clúster S1 (Apruebo)
    terminos_s1 = terminos_filtrados[terminos_filtrados['Cluster'] == 'S1_Apruebo'][['Termino', 'Score_TFIDF_Avg']].head(10).reset_index(drop=True)
    nodos_s1 = nodos_filtrados[nodos_filtrados['Cluster'] == 'S1'][['Nodo', 'Grado']].head(5).reset_index(drop=True)
    terminos_s1.columns = [f'Top 10 Términos (S1_{ngram_tipo})', 'Score']
    nodos_s1.columns = ['Top 5 Nodos (S1)', 'Grado']
    
    # Clúster S2 (Rechazo)
    terminos_s2 = terminos_filtrados[terminos_filtrados['Cluster'] == 'S2_Rechazo'][['Termino', 'Score_TFIDF_Avg']].head(10).reset_index(drop=True)
    nodos_s2 = nodos_filtrados[nodos_filtrados['Cluster'] == 'S2'][['Nodo', 'Grado']].head(5).reset_index(drop=True)
    terminos_s2.columns = [f'Top 10 Términos (S2_{ngram_tipo})', 'Score']
    nodos_s2.columns = ['Top 5 Nodos (S2)', 'Grado']
    
    # Unir los dataframes para una visualización lado a lado
    df_display_terminos = pd.concat([terminos_s1, terminos_s2], axis=1).fillna('')
    df_display_nodos = pd.concat([nodos_s1, nodos_s2], axis=1).fillna('')
    
    print("\n--- ANÁLISIS DE TÉRMINOS ---")
    print(df_display_terminos.to_string())
    
    print("\n\n--- ANÁLISIS DE NODOS CENTRALES ---")
    print(df_display_nodos.to_string())
    print("\n\n")


# Celda 3: Iterar y Mostrar Todos los Resultados

# Obtener listas únicas para iterar
datasets = df_terminos['Dataset'].unique()
algoritmos = df_terminos['Algoritmo'].unique()
ngram_tipos = df_terminos['Tipo_Ngram'].unique()

for dataset in datasets:
    for algoritmo in algoritmos:
        for ngram_tipo in ngram_tipos:
            mostrar_analisis_completo(df_terminos, df_nodos, dataset, algoritmo, ngram_tipo)

Análisis para: DATASET=Plebiscito_PRE | ALGORITMO=Eigensign | N-GRAMS=Unigramas

--- ANÁLISIS DE TÉRMINOS ---
Empty DataFrame
Columns: [Top 10 Términos (S1_Unigramas), Score, Top 10 Términos (S2_Unigramas), Score]
Index: []


--- ANÁLISIS DE NODOS CENTRALES ---
   Top 5 Nodos (S1)  Grado   Top 5 Nodos (S2)  Grado
0     javier macaya    2.0      gabriel boric    298
1       felipe kast    1.0    giorgio jackson     95
2  jorge alessandri    1.0     camila vallejo     80
3                           michelle bachelet     54
4                                izkia siches     52



Análisis para: DATASET=Plebiscito_PRE | ALGORITMO=Eigensign | N-GRAMS=Bigramas

--- ANÁLISIS DE TÉRMINOS ---
Empty DataFrame
Columns: [Top 10 Términos (S1_Bigramas), Score, Top 10 Términos (S2_Bigramas), Score]
Index: []


--- ANÁLISIS DE NODOS CENTRALES ---
   Top 5 Nodos (S1)  Grado   Top 5 Nodos (S2)  Grado
0     javier macaya    2.0      gabriel boric    298
1       felipe kast    1.0    giorgio jackson     95

In [3]:
import pandas as pd

# Define las rutas a tus archivos de resultados
ruta_core = '../results/comparacion_metricas_core.csv'
ruta_paper = '../results/comparacion_metricas_paper.csv'
# Carga los CSV en DataFrames
df_core = pd.read_csv(ruta_core)
df_paper = pd.read_csv(ruta_paper)

df_paper.style \
    .format({
        "POL": "{:.3f}", "BA-POL": "{:.3f}", "BAL": "{:.2%}",
        "MAC": "{:.3f}", "MAO": "{:.3f}", "DENS": "{:.3f}",
        "ISO": "{:.3f}", "CC+": "{:.3f}", "CC-": "{:.3f}"
    }) \
    .background_gradient(
        cmap='RdYlGn',  # Rojo (bajo) a Verde (alto)
        subset=["POL", "BA-POL", "BAL", "ISO", "CC+", "CC-"]
    ) \
    .set_caption("🚀 Tabla 2: Métricas Detalladas")

,Dataset,Algoritmo,POL,BA-POL,BAL,SIZE,ISO,CC+,CC-,CC_intra_S1,CC_intra_S2
0,Plebiscito PRE,Eigensign (Determinista),5.742,4.698,81.82%,120,0.176,0.635,0.880,0.291981,0.581879
1,Plebiscito PRE,Random Eigensign (Probabilístico),2.373,1.727,72.77%,660,0.329,0.603,0.800,0.112295,0.357895
2,Plebiscito PRE,Local Search (beta=0.01),4.631,3.643,78.67%,536,0.452,0.594,0.803,0.248669,0.404152
3,Plebiscito PRE,Local Search (beta=0.005),3.850,3.702,96.15%,816,0.527,0.620,0.798,0.370160,0.208573
4,Plebiscito PRE,SCG (max_obj),5.988,4.459,74.47%,82,0.140,0.649,0.890,0.601742,0.378076
5,Plebiscito PRE,SCG (randomized),2.240,1.655,73.88%,699,0.339,0.603,0.780,0.332854,0.081166
6,Plebiscito PRE,SCG (bansal),2.100,0.900,42.86%,10,0.013,1.000,0.667,0.795238,1.000000
7,Plebiscito POST,Eigensign (Determinista),5.316,1.418,26.67%,19,0.060,0.881,0.730,0.833333,0.844211
8,Plebiscito POST,Random Eigensign (Probabilístico),1.756,1.741,99.15%,468,0.336,0.574,0.676,0.084100,0.279588
9,Plebiscito POST,Local Search (beta=0.01),3.547,3.547,100.00%,400,0.469,0.556,0.697,0.273097,0.437945


In [4]:
# Celda de Análisis Detallado para el Algoritmo Seleccionado

import pandas as pd

# --- CONFIGURACIÓN ---
# Define el algoritmo que quieres analizar en detalle.
ALGORITMO_SELECCIONADO = 'Local Search (b=0.01)'
# Define los nombres de tus clústeres para facilitar el filtrado.
CLUSTER_S1 = 'S1_Apruebo'
CLUSTER_S2 = 'S2_Rechazo'
# Define cuántos términos de ejemplo quieres mostrar en las listas.
N_TERMINOS_A_MOSTRAR = 15

# --- INICIO DEL ANÁLISIS ---
print("=" * 90)
print(f"🔎 ANÁLISIS DETALLADO DE LA EVOLUCIÓN DEL DISCURSO PARA: '{ALGORITMO_SELECCIONADO}'")
print("=" * 90)

# Celda de Análisis para Presentación (Comparando Clústeres por Período)

import pandas as pd

# --- CONFIGURACIÓN ---
# Define el algoritmo que quieres analizar en detalle.
ALGORITMO_SELECCIONADO = 'Local Search (b=0.01)'
# Define los nombres de tus clústeres para facilitar el filtrado.
CLUSTER_S1 = 'S1'
CLUSTER_S2 = 'S2'
# Define cuántos términos quieres mostrar en las tablas.
N_TOP_TERMINOS = 15
# Define cuántos nodos quieres mostrar en las tablas.
N_TOP_NODOS = 5



# 1. ANÁLISIS DE TÉRMINOS POR TIPO DE N-GRAM
# Itera sobre cada tipo de n-grama para analizarlo por separado.
for ngram_tipo in df_terminos['Tipo_Ngram'].unique():
    print(f"\n\n{'='*30} Análisis para N-Grams: {ngram_tipo} {'='*30}")
    
    # Filtra el DataFrame de términos para el algoritmo y n-gram actual.
    terminos_filtrados = df_terminos[
        (df_terminos['Algoritmo'] == ALGORITMO_SELECCIONADO) & 
        (df_terminos['Tipo_Ngram'] == ngram_tipo)
    ]
    
    # --- TABLA 1: ANÁLISIS PRE-PLEBISCITO ---
    print("\n--- 🗓️  Discurso ANTES del Plebiscito (S1 vs S2) ---\n")
    
    terminos_pre = terminos_filtrados[terminos_filtrados['Dataset'] == 'Plebiscito_PRE']
    
    # Prepara las columnas para la tabla comparativa PRE
    terminos_pre_s1 = terminos_pre[terminos_pre['Cluster'] == CLUSTER_S1][['Termino', 'Score_TFIDF_Avg']].head(N_TOP_TERMINOS).reset_index(drop=True)
    terminos_pre_s2 = terminos_pre[terminos_pre['Cluster'] == CLUSTER_S2][['Termino', 'Score_TFIDF_Avg']].head(N_TOP_TERMINOS).reset_index(drop=True)
    
    df_display_pre = pd.concat([
        terminos_pre_s1.rename(columns={'Termino': f'Top Términos ({CLUSTER_S1})', 'Score_TFIDF_Avg': 'Score'}),
        terminos_pre_s2.rename(columns={'Termino': f'Top Términos ({CLUSTER_S2})', 'Score_TFIDF_Avg': 'Score'})
    ], axis=1).fillna('')
    
    print(df_display_pre.to_string())

    
    # --- TABLA 2: ANÁLISIS POST-PLEBISCITO ---
    print("\n\n--- 🗓️  Discurso DESPUÉS del Plebiscito (S1 vs S2) ---\n")
    
    terminos_post = terminos_filtrados[terminos_filtrados['Dataset'] == 'Plebiscito_POST']
    
    # Prepara las columnas para la tabla comparativa POST
    terminos_post_s1 = terminos_post[terminos_post['Cluster'] == CLUSTER_S1][['Termino', 'Score_TFIDF_Avg']].head(N_TOP_TERMINOS).reset_index(drop=True)
    terminos_post_s2 = terminos_post[terminos_post['Cluster'] == CLUSTER_S2][['Termino', 'Score_TFIDF_Avg']].head(N_TOP_TERMINOS).reset_index(drop=True)

    df_display_post = pd.concat([
        terminos_post_s1.rename(columns={'Termino': f'Top Términos ({CLUSTER_S1})', 'Score_TFIDF_Avg': 'Score'}),
        terminos_post_s2.rename(columns={'Termino': f'Top Términos ({CLUSTER_S2})', 'Score_TFIDF_Avg': 'Score'})
    ], axis=1).fillna('')

    print(df_display_post.to_string())


# 2. ANÁLISIS DE NODOS CENTRALES (ACTORES MÁS RELEVANTES)
# Este análisis ya tiene el formato correcto, comparando S1 vs S2 por período.
print(f"\n\n{'='*30} Análisis de Nodos Centrales {'='*30}\n")

# Filtra el DataFrame de nodos para el algoritmo seleccionado.
nodos_filtrados = df_nodos[df_nodos['Algoritmo'] == ALGORITMO_SELECCIONADO]

# --- NODOS PRE-PLEBISCITO ---
nodos_pre = nodos_filtrados[nodos_filtrados['Dataset'] == 'Plebiscito_PRE']
nodos_pre_s1 = nodos_pre[nodos_pre['Cluster'] == CLUSTER_S1][['Nodo', 'Grado']].head(N_TOP_NODOS).reset_index(drop=True)
nodos_pre_s2 = nodos_pre[nodos_pre['Cluster'] == CLUSTER_S2][['Nodo', 'Grado']].head(N_TOP_NODOS).reset_index(drop=True)
df_display_nodos_pre = pd.concat([
    nodos_pre_s1.rename(columns={'Nodo': f'Nodos PRE ({CLUSTER_S1})', 'Grado': 'Grado S1'}),
    nodos_pre_s2.rename(columns={'Nodo': f'Nodos PRE ({CLUSTER_S2})', 'Grado': 'Grado S2'})
], axis=1).fillna('')

print("--- 👥 Actores más relevantes ANTES del plebiscito ---")
print(df_display_nodos_pre.to_string())


# --- NODOS POST-PLEBISCITO ---
nodos_post = nodos_filtrados[nodos_filtrados['Dataset'] == 'Plebiscito_POST']
nodos_post_s1 = nodos_post[nodos_post['Cluster'] == CLUSTER_S1][['Nodo', 'Grado']].head(N_TOP_NODOS).reset_index(drop=True)
nodos_post_s2 = nodos_post[nodos_post['Cluster'] == CLUSTER_S2][['Nodo', 'Grado']].head(N_TOP_NODOS).reset_index(drop=True)
df_display_nodos_post = pd.concat([
    nodos_post_s1.rename(columns={'Nodo': f'Nodos POST ({CLUSTER_S1})', 'Grado': 'Grado S1'}),
    nodos_post_s2.rename(columns={'Nodo': f'Nodos POST ({CLUSTER_S2})', 'Grado': 'Grado S2'})
], axis=1).fillna('')

print("\n--- 👥 Actores más relevantes DESPUÉS del plebiscito ---")
print(df_display_nodos_post.to_string())

# 1. ANÁLISIS DE TÉRMINOS POR TIPO DE N-GRAM
# Itera sobre cada tipo de n-grama para analizarlo por separado.
for ngram_tipo in df_terminos['Tipo_Ngram'].unique():
    print(f"\n\n{'='*30} Análisis para N-Grams: {ngram_tipo} {'='*30}\n")
    
    # Filtra el DataFrame de términos para el algoritmo y n-gram actual.
    terminos_filtrados = df_terminos[
        (df_terminos['Algoritmo'] == ALGORITMO_SELECCIONADO) & 
        (df_terminos['Tipo_Ngram'] == ngram_tipo)
    ]
    
    # Separa los DataFrames de términos para el período PRE y POST.
    terminos_pre = terminos_filtrados[terminos_filtrados['Dataset'] == 'Plebiscito_PRE']
    terminos_post = terminos_filtrados[terminos_filtrados['Dataset'] == 'Plebiscito_POST']
    
    # Itera sobre cada clúster para analizar su discurso.
    for cluster_name in [CLUSTER_S1, CLUSTER_S2]:
        print(f"\n--- 🗣️ Evolución del Discurso del Clúster: {cluster_name} ---\n")
        
        # Usa sets para comparar eficientemente los términos PRE y POST.
        # Los sets contienen los términos más importantes para cada período.
        set_terminos_pre = set(terminos_pre[terminos_pre['Cluster'] == cluster_name]['Termino'])
        set_terminos_post = set(terminos_post[terminos_post['Cluster'] == cluster_name]['Termino'])
        
        # --- Cálculo de la evolución ---
        # Intersección: Términos que aparecen en ambos sets.
        terminos_mantenidos = list(set_terminos_pre.intersection(set_terminos_post))
        # Diferencia: Términos en PRE que no están en POST.
        terminos_perdidos = list(set_terminos_pre - set_terminos_post)
        # Diferencia: Términos en POST que no están en PRE.
        terminos_nuevos = list(set_terminos_post - set_terminos_pre)
        
        # --- Impresión de resultados ---
        print(f"✅ Términos Mantenidos (temas centrales persistentes):")
        print(f"   {terminos_mantenidos[:N_TERMINOS_A_MOSTRAR]}\n")
        
        print(f"❌ Términos Perdidos (temas que perdieron relevancia post-plebiscito):")
        print(f"   {terminos_perdidos[:N_TERMINOS_A_MOSTRAR]}\n")
        
        print(f"✨ Términos Nuevos (temas emergentes post-plebiscito):")
        print(f"   {terminos_nuevos[:N_TERMINOS_A_MOSTRAR]}\n")

# 2. ANÁLISIS DE NODOS CENTRALES (ACTORES MÁS RELEVANTES)
# Este análisis es independiente del tipo de n-gram, por lo que se hace una sola vez.
print(f"\n\n{'='*30} Análisis de Nodos Centrales {'='*30}\n")

# Filtra el DataFrame de nodos para el algoritmo seleccionado.
nodos_filtrados = df_nodos[df_nodos['Algoritmo'] == ALGORITMO_SELECCIONADO]

# --- NODOS PRE-PLEBISCITO ---
nodos_pre = nodos_filtrados[nodos_filtrados['Dataset'] == 'Plebiscito_PRE']
nodos_pre_s1 = nodos_pre[nodos_pre['Cluster'] == CLUSTER_S1][['Nodo', 'Grado']].head().reset_index(drop=True)
nodos_pre_s2 = nodos_pre[nodos_pre['Cluster'] == CLUSTER_S2][['Nodo', 'Grado']].head().reset_index(drop=True)
df_display_pre = pd.concat([
    nodos_pre_s1.rename(columns={'Nodo': f'Nodos PRE ({CLUSTER_S1})', 'Grado': 'Grado S1'}),
    nodos_pre_s2.rename(columns={'Nodo': f'Nodos PRE ({CLUSTER_S2})', 'Grado': 'Grado S2'})
], axis=1).fillna('')

print("--- 👥 Actores más relevantes ANTES del plebiscito ---")
print(df_display_pre.to_string())


# --- NODOS POST-PLEBISCITO ---
nodos_post = nodos_filtrados[nodos_filtrados['Dataset'] == 'Plebiscito_POST']
nodos_post_s1 = nodos_post[nodos_post['Cluster'] == CLUSTER_S1][['Nodo', 'Grado']].head().reset_index(drop=True)
nodos_post_s2 = nodos_post[nodos_post['Cluster'] == CLUSTER_S2][['Nodo', 'Grado']].head().reset_index(drop=True)
df_display_post = pd.concat([
    nodos_post_s1.rename(columns={'Nodo': f'Nodos POST ({CLUSTER_S1})', 'Grado': 'Grado S1'}),
    nodos_post_s2.rename(columns={'Nodo': f'Nodos POST ({CLUSTER_S2})', 'Grado': 'Grado S2'})
], axis=1).fillna('')

print("\n--- 👥 Actores más relevantes DESPUÉS del plebiscito ---")
print(df_display_post.to_string())

🔎 ANÁLISIS DETALLADO DE LA EVOLUCIÓN DEL DISCURSO PARA: 'Local Search (b=0.01)'


============================== Análisis para N-Grams: Unigramas ==============================

--- 🗓️  Discurso ANTES del Plebiscito (S1 vs S2) ---

   Top Términos (S1)    Score Top Términos (S2)    Score
0            rechazo  0.09609             boric  0.06406
1                udi  0.07482           apruebo  0.05884
2       constitución  0.07195      constitución  0.05847
3            senador  0.06861        plebiscito  0.05714
4              carta  0.06344             nueva  0.05484
5         convención  0.06210           rechazo  0.05002
6              lagos  0.06195    constitucional  0.04423
7              texto  0.06146        septiembre  0.04253
8     constitucional  0.06124           gabriel  0.04232
9              nueva  0.05634          ministra  0.04077
10             vamos  0.05452         propuesta  0.03980
11           apruebo  0.05429           proceso  0.03637
12        plebiscito  0.053

## 🎯 Conclusiones del Análisis de Términos Distintivos (TF-IDF)

* **Identificación de Polos:** El análisis de actores y términos confirma que el **Clúster S1** corresponde al polo del **"Rechazo / Oposición"** y el **Clúster S2** al **"Apruebo / Oficialismo"**.

### ⚡ Discurso PRE-Plebiscito: Polarización Clara
* **S1 (Rechazo):** Su lenguaje es de oposición directa. Los términos más distintivos son **"rechazo"**, **"udi"**, **"senador"** y los nombres de figuras críticas al proceso como **"ricardo lagos"**.
* **S2 (Apruebo):** Su lenguaje se centra en el gobierno y la propuesta. Los términos clave son **"boric"**, **"apruebo"** y **"nueva constitución"**.
* **Resumen PRE:** Una clara división entre un polo que ataca y se define por la negativa, y otro que defiende el proyecto desde el oficialismo.

### 🏛️ Discurso POST-Plebiscito: Del Voto al Congreso
El eje de la conversación se traslada de la campaña pública a la negociación institucional.
* **S1 (Oposición):** Su discurso evoluciona. Deja de ser solo "Rechazo" y pasa a hablar del **"nuevo proceso"**, el **"acuerdo"** y el rol de la **"cámara de diputados"**. Actores como **Karol Cariola** se vuelven centrales, reflejando el inicio de las negociaciones en el Congreso.
* **S2 (Oficialismo):** Su discurso se vuelve reactivo. Aunque **"boric"** sigue siendo central, emergen con fuerza términos como **"gabinete"**, **"cambio"** y **"mario marcel"**, reflejando la respuesta a la crisis y la nueva prioridad económica.
* **Resumen POST:** La temática se vuelve más compleja y centrada en la "cocina" política.

---
### 🚀 Conclusión Principal:
**El análisis TF-IDF muestra una transición clara: de una polarización pública y electoral ("Apruebo vs. Rechazo") a una negociación institucional y centrada en el Congreso, donde los discursos de ambos polos se reconfiguran en torno al "nuevo proceso constituyente".**

In [51]:
# Celda para Visualizar los Resultados del Topic Modeling

import pandas as pd
import os

# --- 1. CARGAR LOS RESULTADOS ---
# Asegúrate de que la ruta sea correcta.
results_dir = '../results'
path_temas = os.path.join(results_dir, 'analisis_topic_modeling.csv')

try:
    df_temas = pd.read_csv(path_temas, sep=';')
    print("✅ Archivo con resultados de Topic Modeling cargado exitosamente.")
except FileNotFoundError:
    print("❌ ERROR: No se encontró el archivo 'analisis_topic_modeling.csv'.")
    print("Asegúrate de haber ejecutado el script 'ejecutar_analisis_temas.py' primero desde tu terminal.")

# --- 2. FUNCIÓN PARA MOSTRAR LOS TEMAS DE FORMA ORDENADA ---
def mostrar_temas_por_escenario(df, dataset, cluster):
    """Filtra el DataFrame y muestra los temas para un escenario específico."""
    
    print("=" * 80)
    print(f"🗣️  TEMAS DE DISCUSIÓN PARA: {dataset} | {cluster}")
    print("=" * 80)
    
    # Filtra los datos para el escenario actual.
    df_filtrado = df[(df['Dataset'] == dataset) & (df['Cluster'] == cluster)]
    
    if df_filtrado.empty:
        print("No se encontraron temas para este escenario.")
        return
        
    # Muestra los temas encontrados, excluyendo el tema -1 (outliers).
    temas_principales = df_filtrado[df_filtrado['Topic'] != -1]
    
    # Selecciona y renombra columnas para una visualización limpia.
    display_cols = {
        'Topic': 'ID Tema',
        'Count': 'N° Noticias',
        'Name': 'Nombre del Tema',
        'Representation': 'Palabras Clave'
    }
    
    # Imprime la tabla formateada.
    print(temas_principales[display_cols.keys()].rename(columns=display_cols).to_string(index=False))

# --- 3. MOSTRAR LOS RESULTADOS PARA CADA ESCENARIO ---
if 'df_temas' in locals():
    mostrar_temas_por_escenario(df_temas, 'Plebiscito_PRE', 'S1')
    print("\n")
    mostrar_temas_por_escenario(df_temas, 'Plebiscito_PRE', 'S2')
    print("\n")
    mostrar_temas_por_escenario(df_temas, 'Plebiscito_POST', 'S1')
    print("\n")
    mostrar_temas_por_escenario(df_temas, 'Plebiscito_POST', 'S2')

✅ Archivo con resultados de Topic Modeling cargado exitosamente.
🗣️  TEMAS DE DISCUSIÓN PARA: Plebiscito_PRE | S1
No se encontraron temas para este escenario.


🗣️  TEMAS DE DISCUSIÓN PARA: Plebiscito_PRE | S2
 ID Tema  N° Noticias                                 Nombre del Tema                                                                                                                         Palabras Clave
       0          153         0_jackson_ministro_segpres_constitución              ['jackson', 'ministro', 'segpres', 'constitución', 'giorgio', 'nueva', 'boric', 'secretario', 'plebiscito', 'propuestas']
       1          124              1_boric_constitución_nueva_proceso              ['boric', 'constitución', 'nueva', 'proceso', 'plebiscito', 'rechazo', 'vamos', 'mandatario', 'constituyente', 'gabriel']
       2           88    2_contraloría_ministra_vallejo_investigación ['contraloría', 'ministra', 'vallejo', 'investigación', 'plebiscito', 'intervencionismo', 'moneda', 'sich

In [52]:
# Celda Final para Visualizar Resultados Combinados (Temas y Nodos)

import pandas as pd
import os

# --- 1. CONFIGURACIÓN ---
# Define el algoritmo que quieres analizar. Debe ser el mismo que usaste en el script.
ALGORITMO_SELECCIONADO = 'Local Search (b=0.01)'
# Define cuántos nodos mostrar.
N_TOP_NODOS = 5

# --- 2. CARGAR TODOS LOS RESULTADOS NECESARIOS ---
results_dir = '../results'
path_temas = os.path.join(results_dir, 'analisis_topic_modeling.csv')
path_nodos = os.path.join(results_dir, 'exp_analisis_nodos.csv')

try:
    df_temas = pd.read_csv(path_temas, sep=';')
    df_nodos = pd.read_csv(path_nodos, sep=';')
    print("✅ Archivos de Temas y Nodos cargados exitosamente.")
except FileNotFoundError as e:
    print(f"❌ ERROR: No se encontró un archivo de resultados: {e}")
    print("Asegúrate de haber ejecutado los scripts 'ejecutar_analisis_clivajes...' y 'ejecutar_analisis_temas...' primero.")

# --- 3. FUNCIÓN MEJORADA PARA MOSTRAR TODA LA INFORMACIÓN ---
def mostrar_analisis_completo_por_escenario(df_temas, df_nodos, dataset, cluster):
    """
    Filtra y muestra los temas de discusión Y los actores principales
    para un escenario específico.
    """
    
    print("=" * 80)
    print(f"🔎 ANÁLISIS PARA: {dataset} | {cluster}")
    print("=" * 80)
    
    # --- Mostrar Temas de Discusión ---
    df_temas_filtrado = df_temas[(df_temas['Dataset'] == dataset) & (df_temas['Cluster'] == cluster)]
    
    if df_temas_filtrado.empty:
        print("No se encontraron temas de discusión para este escenario.")
    else:
        print("🗣️ TEMAS DE DISCUSIÓN PRINCIPALES:")
        temas_principales = df_temas_filtrado[df_temas_filtrado['Topic'] != -1]
        display_cols = {
            'Topic': 'ID', 'Count': 'N° Docs', 'Name': 'Nombre del Tema', 'Representation': 'Palabras Clave'
        }
        print(temas_principales[display_cols.keys()].rename(columns=display_cols).to_string(index=False))

    print("-" * 80) # Separador visual

    # --- Mostrar Nodos Principales ---
    # Filtra el df de nodos para el mismo escenario (algoritmo, dataset, cluster)
    df_nodos_filtrado = df_nodos[
        (df_nodos['Algoritmo'] == ALGORITMO_SELECCIONADO) &
        (df_nodos['Dataset'] == dataset) & 
        (df_nodos['Cluster'] == cluster)
    ]

    if df_nodos_filtrado.empty:
        print("No se encontraron nodos centrales para este escenario.")
    else:
        print(f"👥 ACTORES PRINCIPALES (Top {N_TOP_NODOS} con mayor grado):")
        nodos_principales = df_nodos_filtrado[['Nodo', 'Grado']].head(N_TOP_NODOS)
        print(nodos_principales.to_string(index=False))


# --- 4. MOSTRAR LOS RESULTADOS COMPLETOS PARA CADA ESCENARIO ---
if 'df_temas' in locals() and 'df_nodos' in locals():
    # Obtiene los clústeres y datasets del archivo de temas para iterar
    escenarios = df_temas[['Dataset', 'Cluster']].drop_duplicates().to_dict('records')
    
    for escenario in escenarios:
        mostrar_analisis_completo_por_escenario(df_temas, df_nodos, escenario['Dataset'], escenario['Cluster'])
        print("\n")

✅ Archivos de Temas y Nodos cargados exitosamente.
🔎 ANÁLISIS PARA: Plebiscito_PRE | S2
🗣️ TEMAS DE DISCUSIÓN PRINCIPALES:
 ID  N° Docs                                 Nombre del Tema                                                                                                                         Palabras Clave
  0      153         0_jackson_ministro_segpres_constitución              ['jackson', 'ministro', 'segpres', 'constitución', 'giorgio', 'nueva', 'boric', 'secretario', 'plebiscito', 'propuestas']
  1      124              1_boric_constitución_nueva_proceso              ['boric', 'constitución', 'nueva', 'proceso', 'plebiscito', 'rechazo', 'vamos', 'mandatario', 'constituyente', 'gabriel']
  2       88    2_contraloría_ministra_vallejo_investigación ['contraloría', 'ministra', 'vallejo', 'investigación', 'plebiscito', 'intervencionismo', 'moneda', 'siches', 'campaña', 'ministerio']
  3       46            3_acuerdo_constitución_apruebo_nueva                    ['acuerdo', '

## 🎯 Conclusiones del Análisis de Topic Modeling (Clúster Oficialismo/Apruebo)

* **Identificación del Clúster:** Los actores principales (Boric, Jackson, Vallejo, Bachelet) confirman que el clúster analizado (`S2`) corresponde al polo del **Oficialismo / Apruebo**.

### 📝 Agenda PRE-Plebiscito: Campaña "Multi-Frente"
El discurso antes de la elección era amplio y cubría múltiples frentes simultáneamente:
* **Promoción:** Impulso del "proceso constituyente" y la "nueva constitución".
* **Legitimación:** Búsqueda de respaldos clave, con temas específicos dedicados a las figuras de **Michelle Bachelet** y **Ricardo Lagos**.
* **Defensa:** Gestión de controversias, como las acusaciones de "intervencionismo electoral" investigadas por la **Contraloría**.
* **Contingencia:** Seguimiento de las encuestas ("Cadem") y gestión de problemas de gobierno como el "estado de excepción".

### 📉 Agenda POST-Plebiscito: Contracción y Gestión de Crisis
Tras la derrota, la agenda se contrae y se enfoca en la crisis inmediata:
* **Daño Político:** El tema dominante es el **"cambio de gabinete"**, con la salida de ministros como Izkia Siches y Giorgio Jackson.
* **Estabilidad Económica:** Emerge un nuevo y fuerte discurso centrado en el Ministro de Hacienda, **Mario Marcel**, para calmar a los mercados e inversionistas.
* **Rearticulación Política:** La conversación se traslada al Congreso, con la aparición de la nueva Ministra Segpres, **Ana Lya Uriarte**, como figura clave en la negociación del "nuevo proceso".

---
### 🚀 Conclusión Principal:
**El discurso del oficialismo pasó de una narrativa de campaña expansiva y multifacética a una de gestión de crisis, enfocada en la reestructuración política interna y en proyectar estabilidad económica.**